In [21]:
#imports

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import pathlib
import gc

In [57]:
class_names = ['NonDemented', 'Demented']
num_classes = len(class_names)
batch_size = 32
input_shape = (176, 208, 3)
AUTOTUNE = tf.data.experimental.AUTOTUNE

def one_hot_label(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label

def construct_model(noise, pretrained, dense0, dense1):
    model = keras.models.Sequential([
        keras.Input(input_shape),
        layers.GaussianNoise(noise),
        pretrained,
        layers.Flatten(),
        #layers.Dropout(0.4),
        layers.Dense(dense0, activation='relu'),
        #layers.Dropout(0.3),
        layers.Dense(dense1, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

In [58]:
train_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/train')
test_dir = pathlib.Path('../Project/Alzheimer_s Dataset_binary/test')

train_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        labels='inferred',
        class_names=class_names,
        batch_size=batch_size,
        image_size=input_shape[0:2])

test_dataset = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        labels='inferred',
        class_names=class_names,
        batch_size=batch_size,
        image_size=input_shape[0:2])

Found 5121 files belonging to 2 classes.
Found 1279 files belonging to 2 classes.


In [59]:
train_dataset = train_dataset.map(one_hot_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(one_hot_label, num_parallel_calls=AUTOTUNE)

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [42]:
vgg = keras.applications.VGG19(
        include_top=False,
        input_shape=input_shape,
        pooling=max)

for layer in vgg.layers:
    layer.trainable = False
#vgg.layers[17].trainable = True
#vgg.layers[18].trainable = True
#vgg.layers[19].trainable = True
#vgg.layers[20].trainable = True

vgg.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 176, 208, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 176, 208, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 176, 208, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 88, 104, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 88, 104, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 88, 104, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 44, 52, 128)       0     

In [28]:
epochs = 30

accuracy_list = []


for dense2 in range(30):
    keras.backend.clear_session()
    gc.collect()
    model = keras.models.Sequential([
                keras.Input(input_shape),
                layers.GaussianNoise(0.2*dense2),
                vgg,
                layers.Flatten(),
                layers.Dense(4, activation='relu'),
                layers.Dense(4, activation='relu'),
                layers.Dense(16, activation='relu'),
                layers.Dense(num_classes, activation='softmax'),
             ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    history = model.fit(train_dataset, steps_per_epoch=161, shuffle=True, epochs=epochs, validation_data=test_dataset, verbose=0)
    accuracy_list.append(history.history['val_accuracy'][epochs-1])
    del model
            
print(accuracy_list)
max_value = max(accuracy_list)
index = accuracy_list.index(max_value)
print(f'\nnoise={0.2*index}, dense0={2**(index//16+2)}, dense1={2**(index//4%4+2)}, dense2={2**(index%4+2)}, acc={max_value}')

[0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.7154026627540588, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.7099296450614929, 0.4996090829372406, 0.6770914793014526, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406, 0.4996090829372406]

noise=0.6000000000000001, dense0=4, dense1=4, dense2=16 acc=0.7154026627540588


In [48]:
#### Create a transfer learning model ####

epochs = 30
keras.backend.clear_session()
gc.collect()
model = keras.models.Sequential([
        keras.Input(input_shape),
        layers.GaussianNoise(0.0),
        vgg,
        layers.Flatten(),
        layers.Dense(8, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
history = model.fit(train_dataset, steps_per_epoch=161, shuffle=True, epochs=epochs, validation_data=test_dataset)

Epoch 1/30
 42/161 [======>.......................] - ETA: 16s - loss: 0.8165 - accuracy: 0.6429

KeyboardInterrupt: 

In [ ]:
#### Create a CNN from scratch ####

epochs = 30
keras.backend.clear_session()
gc.collect()
model = keras.models.Sequential([
        keras.Input(input_shape),
        #layers.GaussianNoise(0.5),
        #Creates the convolutional layer
        layers.Conv2D(16, kernel_size=(3,3), activation='relu'),
        layers.Conv2D(16, kernel_size=(3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        
        layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
        layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        
        #layers.SeparableConv2D(64, kernel_size=(3,3), activation='relu'),
        #layers.SeparableConv2D(64, kernel_size=(3,3), activation='relu'),
        #layers.BatchNormalization(),
        #layers.MaxPool2D(),
        
        #layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
        #layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
        #layers.BatchNormalization(),
        #layers.MaxPool2D(),
        
        #layers.Conv2D(256, kernel_size=(3,3), activation='relu'),
        #layers.Conv2D(256, kernel_size=(3,3), activation='relu'),
        #layers.BatchNormalization(),
        #layers.MaxPool2D(),
    
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
history = model.fit(train_dataset, steps_per_epoch=161, shuffle=True, epochs=epochs, validation_data=test_dataset)

Epoch 1/30
161/161 [==============================] - 8s 44ms/step - loss: 0.7498 - accuracy: 0.5048 - val_loss: 0.6932 - val_accuracy: 0.5004
Epoch 2/30
161/161 [==============================] - 7s 43ms/step - loss: 0.6932 - accuracy: 0.4901 - val_loss: 0.6931 - val_accuracy: 0.5004
Epoch 3/30
161/161 [==============================] - 7s 43ms/step - loss: 0.6932 - accuracy: 0.4911 - val_loss: 0.6931 - val_accuracy: 0.4996


In [ ]:
plt.title('VGG19 Transfer Learning w/ Gaussian(1) Accuracy with Dropout')

plt.plot(history.history['accuracy'], label = "training acc.")
plt.plot(history.history['val_accuracy'], label = "validation acc.")
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training acc.', 'validation acc.'], loc='upper left')

plt.savefig("../Project/Result2/VGG19_MODEL_Pre_1_ACCURACY_adam_dropout.png")
plt.show()
